# station_zdz  --- 的后端模块设计

In [10]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
class station_zdz:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.rs = redis.Redis(host='127.0.0.1', port=6379)
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
    def single_station(self,station):
        '''获取单站数据并解析'''
        # 数据库中读取单站数据并解析
        sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.lat,station.lon,Ri,T,V,fFy,dFy 
        from Tab_AM_M as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
        where (ta.IIiii='{station}' and tTime between '{start_time}' and '{end_time}')  order by tTime  """
        rsql = sql.format(start_time=self.start,end_time=self.end,station=station)
        df = pd.read_sql(rsql , con=self.conn)
        return df
    def upload2_redis_24h(self):
        '''根据date_type向redis中传输数据'''
        tables = ['TAB_Aws2019','TAB_Mws2019']
        data_list = []
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        end = today.strftime('%Y-%m-%d %H:%M:%S')
        offset = dtt.timedelta(days=-1)
        start = (today + offset).strftime('%Y-%m-%d %H:%M:%S') 
        # 数据层
        start = "2019" + start[4:]
        end = "2019" + end[4:]
        for table in tables:
            sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.lat,station.lon,RR,T,VV,fFy,dFy 
            from {table} as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
            where (tTime between '{start}' and '{end}') order by tTime """
            rsql = sql.format(start=start,end=end,table=table)
            df = pd.read_sql(rsql , con=self.conn) 
            data_list.append(df)
        station_all = pd.concat(data_list)
        data = {
            "start":start,
            "end":end,
            "data_class":"hours24",
            "data":station_all
        }
        self.rs.set("hours24", pickle.dumps(data))
    def upload2_redis_3h(self):
        '''根据date_type向redis中传输数据'''
        table = 'Tab_AM_M'
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        end = today.strftime('%Y-%m-%d %H:%M:%S')
        offset = dtt.timedelta(hours=-3)
        start = (today + offset).strftime('%Y-%m-%d %H:%M:%S') 
        # 数据层
        start = '2019-08-08 06:00:00'
        end = '2019-08-08 09:00:00' 
        sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.lat,station.lon,Ri,T,V,fFy,dFy 
        from {table} as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
        where (tTime between '{start}' and '{end}') order by tTime """
        rsql = sql.format(start=start,end=end,table=table)
        df = pd.read_sql(rsql , con=self.conn) 
        data = {
            "start":start,
            "end":end,
            "data_class":"hours03",
            "data":df
        }
        self.rs.set("hours03", pickle.dumps(data))
    def decode_dataframe(self,dataframe,decode_type):
        '''根据date_type向redis中获取数据'''
        date_type = "hours24"  
        table_data = []
        data = pickle.loads(self.rs.get(date_type))
        station_all = data['data']  
        grouped_IIiii = station_all.groupby('IIiii')
        for i in grouped_IIiii.size().index:
            single = grouped_IIiii.get_group(i)
            single['tTime'] = pd.to_datetime(single['tTime'])
            end_time = single['tTime'].iloc[-1]
            offset = dtt.timedelta(hours=-decode_type)
            start_time = (end_time + offset)
            start_str = (end_time + offset).strftime('%Y-%m-%d %H:%M:%S')
            end_str = end_time.strftime('%Y-%m-%d %H:%M:%S')
            single_data = single[single['tTime']>start_str]
            # 制定游戏规则
            RR = single_data[single_data['Ri'] > 0.0]['Ri'].sum()
            tmin = single_data['T'].min()
            tmax = single_data['T'].max()
            vv = single_data[(single_data['V'] >0) & (single_data['V'] < 30000)]['V'].min()
            wind = single_data[single_data['fFy'] > 0]['fFy'].max()
            index =  single_data[single_data['fFy'] == single_data['fFy'].max()].index.tolist()[0]
            deg = single_data['dFy'][index]
            single_dir = {
                "IIiii":str(single_data['IIiii'].iloc[0]),
                "county":str(single_data['county'].iloc[0]),
                "town":str(single_data['Town'].iloc[0]),
                "StationName":str(single_data['StationName'].iloc[0]),
                "fFy":str(wind),
                "dFy":str(deg),
                "RR":str(RR),
                "Tx":str(tmax),
                "Tn":str(tmin),
                "VV":str(vv),
                "lat":str(single_data['lat'].iloc[0]),
                "lon":str(single_data['lon'].iloc[0])
            }
            table_data.append(single_dir)
    def get_redis(self,date_type):
        '''根据date_type向redis中获取数据'''
        data = pickle.loads(self.rs.get(date_type))
        # 解析数据
        return data

start = '2019-08-08 06:00:00'
end = '2019-08-08 18:00:00'  
date_type = 'rr24'
station = '58653'
worker = station_zdz(start,end)
dataframe = None
decode_type = 1
# worker.decode_dataframe(dataframe,decode_type)
worker.single_station(station)
# worker.upload2_redis_3h()
# data = worker.get_redis(date_type)



CPU times: user 173 ms, sys: 20.4 ms, total: 194 ms
Wall time: 6.37 s


,tTime,IIiii,StationName,Province,City,County,Town,lat,lon,Ri,T,V,fFy,dFy
0,2019-08-08 06:00:00,58653,括苍山,浙江,台州,临海市,括苍镇,28.80972,120.9222,0.0,197,-9999,143,274
1,2019-08-08 06:05:00,58653,括苍山,浙江,台州,临海市,括苍镇,28.80972,120.9222,0.0,196,-9999,126,277
2,2019-08-08 06:10:00,58653,括苍山,浙江,台州,临海市,括苍镇,28.80972,120.9222,0.0,198,-9999,126,277
3,2019-08-08 06:15:00,58653,括苍山,浙江,台州,临海市,括苍镇,28.80972,120.9222,0.0,197,-9999,126,277
4,2019-08-08 06:20:00,58653,括苍山,浙江,台州,临海市,括苍镇,28.80972,120.9222,0.0,196,-9999,126,277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2019-08-08 17:40:00,58653,括苍山,浙江,台州,临海市,括苍镇,28.80972,120.9222,10.0,204,-9999,155,236
141,2019-08-08 17:45:00,58653,括苍山,浙江,台州,临海市,括苍镇,28.80972,120.9222,37.0,205,-9999,155,236
142,2019-08-08 17:50:00,58653,括苍山,浙江,台州,临海市,括苍镇,28.80972,120.9222,25.0,205,-9999,155,236
143,2019-08-08 17:55:00,58653,括苍山,浙江,台州,临海市,括苍镇,28.80972,120.9222,44.0,206,-9999,155,236


In [45]:
date_type = 'hours24'
data = worker.get_redis(date_type)

In [58]:
alldata = data['data']
#alldata[alldata['tTime']>'2019-08-08 07:00:00']
alldata

,tTime,IIiii,StationName,Province,City,County,Town,lat,lon,Ri,T,V,fFy,dFy
0,2019-08-08 06:00:00,58467,慈溪,浙江,宁波,慈溪市,白沙路街道,30.200000,121.300000,0.0,274,16272,16,85
1,2019-08-08 06:00:00,58473,嵊山,浙江,舟山,嵊泗县,嵊山镇,30.733333,122.816667,NaN,277,27766,91,109
2,2019-08-08 06:00:00,58546,浦江,浙江,金华,浦江县,浦阳街道,29.466667,119.883333,0.0,259,6430,29,343
3,2019-08-08 06:00:00,58570,普陀,浙江,舟山,普陀区,沈家门街道,29.950000,122.300000,0.0,274,11797,53,102
4,2019-08-08 06:00:00,58654,缙云,浙江,丽水,缙云县,五云街道,28.666667,120.083333,0.0,260,17996,4,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116892,2019-08-08 09:00:00,K9559,勾山,浙江,舟山,普陀区,临城街道,29.983333,122.250000,0.0,304,-9999,78,69
116893,2019-08-08 09:00:00,K9566,衢山(备),浙江,舟山,岱山县,衢山镇,30.450000,122.283333,2.0,285,-9999,91,77
116894,2019-08-08 09:00:00,K9606,大衢渔港,浙江,舟山,岱山县,衢山镇,30.466667,122.283333,1.0,293,-9999,68,53
116895,2019-08-08 09:00:00,K9640,六横东,浙江,舟山,定海区,六横东,29.700000,122.200000,0.0,-9999,20000,-9999,-9999


In [52]:
data['data']['tTime'] = pd.to_datetime(data['data']['tTime'])
data['data'].set_index('tTime', inplace=False)

data['data']('2019-08-08 06:00:00','2019-08-08 07:00:00')

TypeError: Index must be DatetimeIndex

# 以下为测试代码

In [5]:
import datetime as dtt
from datetime import timezone
SHA_TZ = timezone(
    dtt.timedelta(hours=8),
    name='Asia/Shanghai',
)
utc_now = dtt.datetime.utcnow().replace(tzinfo=timezone.utc)
today = utc_now.astimezone(SHA_TZ)
today_str = today.strftime('%Y-%m-%d %H:%M:%S')
offset = dtt.timedelta(seconds=-300)
re_date = (today + offset).strftime('%Y-%m-%d %H:%M:%S') 
re_date

'2023-05-14 17:47:04'

In [5]:
import datetime as dtt
# from datetime import timedelta
# from datetime import timezone

utc_now = dtt.datetime.utcnow().replace(tzinfo=timezone.utc)

SHA_TZ = timezone(
    dtt.timedelta(hours=8),
    name='Asia/Shanghai',
)

# 北京时间
beijing_now = utc_now.astimezone(SHA_TZ)
print(beijing_now)
print(type(beijing_now))

# fmt = '%Y-%m-%d %H:%M:%S'
# now_fmt =beijing_now.strftime(fmt)
# print(now_fmt)
# print(type(now_fmt))


NameError: name 'timezone' is not defined

In [13]:
%%time
data = worker.get_redis(date_type)

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 8.18 ms


In [14]:
data['data']

,station_no,station_province,station_city,station_county,station_town,station_village,station_country,station_name,lon,lat,p_total,t_max,t_min,w_max,w_dir,vis
0,58443,浙江,湖州,长兴县,龙山街道,None,长兴县,长兴,119.900000,31.016667,0.0,342.0,331.0,95.0,118.0,21012.0
1,58444,浙江,杭州,临平区,临平街道,None,临平区,临平,120.283333,30.416667,0.0,315.0,307.0,116.0,158.0,17951.0
2,58446,浙江,湖州,安吉县,昌硕街道,None,安吉县,安吉,119.700000,30.633333,0.0,343.0,322.0,88.0,127.0,26938.0
3,58448,浙江,杭州,临安区,锦南街道,None,临安区,临安,119.733333,30.300000,0.0,327.0,316.0,124.0,130.0,23765.0
4,58449,浙江,杭州,富阳区,富春街道,None,富阳区,富阳,119.950000,30.050000,0.0,328.0,320.0,93.0,357.0,26791.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3063,K9728,浙江,舟山,定海区,岑港街道,None,定海区,大五峙,121.883333,30.216667,0.0,NaN,NaN,130.0,94.0,19800.0
3064,K9729,浙江,舟山,普陀区,沈家门街道,None,普陀区,点灯山,122.233333,29.883333,0.0,NaN,NaN,123.0,73.0,5085.0
3065,K9735,浙江,舟山,普陀区,六横镇,None,普陀区,双塘,122.150000,29.720000,0.0,294.0,285.0,120.0,100.0,13564.0
3066,K9739,浙江,舟山,嵊泗,菜园镇,None,嵊泗,下三横山,122.650000,30.770000,0.0,281.0,277.0,137.0,79.0,NaN


In [12]:
start2 = '2019-08-07 21:00:00'
end2 = '2019-08-08 21:00:00'  
date_type2 = 'rr24'
worker2 = station_zdz(start2,end2)
worker2.upload_redis(date_type2)

data2 = worker2.get_redis(date_type2)

In [20]:
start2 = '2019-08-07 01:00:00'
end2 = '2019-08-08 10:00:00'  
date_type2 = 'rr24'
worker3 = station_zdz(start2,end2)
data3= worker3.get_redis(date_type2)
data3['data']

,tTime,Ri
0,2019-08-07 21:55:00,0.0
1,2019-08-07 22:35:00,0.0
2,2019-08-08 00:00:00,0.0
3,2019-08-07 21:15:00,0.0
4,2019-08-07 21:35:00,0.0
...,...,...
285,2019-08-08 20:40:00,1.0
286,2019-08-08 20:45:00,0.0
287,2019-08-08 20:50:00,0.0
288,2019-08-08 20:55:00,1.0


# 增量同步redis的策略  - 保存一天的数据

In [1]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
from datetime import timezone
class station_zdz:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.rs = redis.Redis(host='127.0.0.1', port=6379)
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
    def current_date(self):
        '''获取单站数据并解析'''
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        end = today.strftime('%Y-%m-%d %H:%M:%S')
        offset = dtt.timedelta(seconds=-300)
        start = (today + offset).strftime('%Y-%m-%d %H:%M:%S')
        daydelay = dtt.timedelta(days=-1)
        daystar = (today + daydelay).strftime('%Y-%m-%d %H:%M:%S')
        return start,end,daystar
    def single_station(self,station):
        '''获取单站数据并解析'''
        # 获取当前时间
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        end = today.strftime('%Y-%m-%d %H:%M:%S')
        offset = dtt.timedelta(days=-1)
        start = (today + offset).strftime('%Y-%m-%d %H:%M:%S')
        # 造假数据
        start = '2019-08-07 20:53:00'
        end = '2019-08-08 20:53:00' 
        yesday = start[0:10] + " 20:00:00"
        today = end[0:10] + " 20:00:00"
        hours = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S').hour
        # 数据库中读取单站数据并解析
        sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.lat,station.lon,Ri,T,V,fFy,dFy 
        from Tab_AM_M as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
        where (ta.IIiii='{station}' and tTime between '{start_time}' and '{end_time}')  order by tTime  """
        rsql = sql.format(start_time=start,end_time=end,station=station)
        data = pd.read_sql(rsql , con=self.conn)
        if hours>=20:
            # 实时数据
            now_data = data[data['tTime']>=today]
            now_data = now_data[now_data['tTime']<=end]
            # 历史数据
            his_data = data[data['tTime']>start]
            his_data = his_data[his_data['tTime']<=today]   
        else:
            # 实时数据
            now_data = data[data['tTime']>=yesday]
            now_data = now_data[now_data['tTime']<=end]
            # 历史数据
            his_data = data[data['tTime']>start]
            his_data = his_data[his_data['tTime']<=yesday]
        # 开始筛选数据种类
        nul_now = [0 for i in range(len(now_data['T'].to_list()))]    
        nul_his =  [0 for i in range(len(his_data['T'].to_list()))]
        value_now =  now_data['T'].to_list()  + nul_his
        value_his = nul_now + his_data['T'].to_list() 
        # 解析数据成两个序列
        return data
    def upload2_redis_Hours(self):
        '''根据date_type向redis中传输数据每五分钟传一个数据，并输出对应的键值对'''
        tables = ['TAB_Aws2019','TAB_Mws2019']
        data_list = []
        start,end,daystar = self.current_date()
        # 修改数据层
        start = "2019" + start[4:]
        end = "2019" + end[4:]
        for table in tables:
            sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.lat,station.lon,RR,T,VV,fFy,dFy 
            from {table} as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
            where (tTime between '{start}' and '{end}') order by tTime """
            rsql = sql.format(start=start,end=end,table=table)
            df = pd.read_sql(rsql , con=self.conn) 
            data_list.append(df)
        station_all = pd.concat(data_list)
        # 合并pandas 
        data_class = "table_hours"
        redis_data = self.get_redis(data_class)
        if not redis_data:
            # 当redis无数据时
            data = {
                "data_class":"table_hours",
                "data":station_all
            }
        else:
            # 当redis有数据时,存储数据并保留最近24小时的
            redis_df = redis_data['data']
            redis_df['tTime'] = pd.to_datetime(redis_df['tTime'])
            remain_df = redis_df[redis_df['tTime']>=daystar]
            output = pd.concat([remain_df,station_all])
            output['tTime'] = pd.to_datetime(output['tTime'])
            output.drop_duplicates(keep='first',inplace=True)
            data = {
               "data_class":"table_hours",
                "data":output
            }
        # 将保留的数据重新存储到redis中
        self.rs.set("table_hours", pickle.dumps(data))
    def upload2_redis_Minutes(self):
        '''根据date_type向redis中传输数据'''
        table = 'Tab_AM_M'
        start,end,daystar = self.current_date()
        # 数据层
        start = '2019-08-08 20:00:00'
        end = '2019-08-09 20:00:00' 
        daystar = '2019-08-08 20:00:00'
        sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.ZoomLevel,station.Type,station.lat,station.lon,Ri,T,V,fFy,dFy 
        from {table} as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
        where (tTime between '{start}' and '{end}') order by tTime """
        rsql = sql.format(start=start,end=end,table=table)
        station_all = pd.read_sql(rsql , con=self.conn) 
        data = {
            "time":end,
            "data_class":"table_minutes",
            "data":station_all
        }
        # 将保留的数据重新存储到redis中
        self.rs.set("table_minutes", pickle.dumps(data))        
    def decode_dataframe(self,dataframe,decode_type):
        '''根据date_type向redis中获取数据'''
        date_type = "hours24"  
        table_data = []
        data = pickle.loads(self.rs.get(date_type))
        station_all = data['data']  
        grouped_IIiii = station_all.groupby('IIiii')
        for i in grouped_IIiii.size().index:
            single = grouped_IIiii.get_group(i)
            single['tTime'] = pd.to_datetime(single['tTime'])
            end_time = single['tTime'].iloc[-1]
            offset = dtt.timedelta(hours=-decode_type)
            start_time = (end_time + offset)
            start_str = (end_time + offset).strftime('%Y-%m-%d %H:%M:%S')
            end_str = end_time.strftime('%Y-%m-%d %H:%M:%S')
            single_data = single[single['tTime']>start_str]
            # 制定游戏规则
            RR = single_data[single_data['Ri'] > 0.0]['Ri'].sum()
            tmin = single_data['T'].min()
            tmax = single_data['T'].max()
            vv = single_data[(single_data['V'] >0) & (single_data['V'] < 30000)]['V'].min()
            wind = single_data[single_data['fFy'] > 0]['fFy'].max()
            index =  single_data[single_data['fFy'] == single_data['fFy'].max()].index.tolist()[0]
            deg = single_data['dFy'][index]
            single_dir = {
                "IIiii":str(single_data['IIiii'].iloc[0]),
                "county":str(single_data['county'].iloc[0]),
                "town":str(single_data['Town'].iloc[0]),
                "StationName":str(single_data['StationName'].iloc[0]),
                "fFy":str(wind),
                "dFy":str(deg),
                "RR":str(RR),
                "Tx":str(tmax),
                "Tn":str(tmin),
                "VV":str(vv),
                "lat":str(single_data['lat'].iloc[0]),
                "lon":str(single_data['lon'].iloc[0])
            }
            table_data.append(single_dir)
    def get_redis(self,date_type):
        '''根据date_type向redis中获取数据'''
        value = self.rs.get(date_type)
        if value:
            data = pickle.loads(value)
        else:
            data = None       
        #data = pickle.loads(self.rs.get(date_type))
        # 解析数据
        return data

start = '2019-08-08 20:00:00'
end = '2019-08-09 20:00:00'  
date_type = 'table_minutes'
station = '58653'
worker = station_zdz(start,end)
dataframe = None
decode_type = 1
# worker.decode_dataframe(dataframe,decode_type)
#data = worker.single_station(station)
worker.upload2_redis_Minutes()
# data = worker.get_redis(date_type)
# data['data']

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


CPU times: user 40.4 s, sys: 1.7 s, total: 42.1 s
Wall time: 24min 28s


In [44]:
%%time
start = '2019-08-07 20:53:00' 
end = '2019-08-08 20:53:00' # now
yesday = start[0:10] + " 20:00:00"
today = end[0:10] + " 20:00:00"
hours = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S').hour
if hours>=20:
    # 实时数据
    now_data = data[data['tTime']>='2019-08-08 20:00:00']
    now_data = now_data[now_data['tTime']<='2019-08-08 20:83:00']
    # 历史数据
    his_data = data[data['tTime']>'2019-08-07 20:53:00']
    his_data = his_data[his_data['tTime']<='2019-08-08 20:00:00']   
else:
    # 实时数据
    now_data = data[data['tTime']>='2019-08-08 20:00:00']
    now_data = now_data[now_data['tTime']<='2019-08-08 20:83:00']
    # 历史数据
    his_data = data[data['tTime']>'2019-08-07 20:53:00']
    his_data = his_data[his_data['tTime']>'2019-08-07 20:00:00']


CPU times: user 10.4 ms, sys: 0 ns, total: 10.4 ms
Wall time: 6.9 ms


In [55]:
%%time
start = '2019-08-07 20:53:00' 
end = '2019-08-08 20:53:00' # now
yesday = start[0:10] + " 20:00:00"
today = end[0:10] + " 20:00:00"
hours = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S').hour
if hours>=20:
    # 实时数据
    now_data = data[data['tTime']>=today]
    now_data = now_data[now_data['tTime']<=end]
    # 历史数据
    his_data = data[data['tTime']>start]
    his_data = his_data[his_data['tTime']<=today]   
else:
    # 实时数据
    now_data = data[data['tTime']>=yesday]
    now_data = now_data[now_data['tTime']<=end]
    # 历史数据
    his_data = data[data['tTime']>start]
    his_data = his_data[his_data['tTime']<=yesday]

nul_now = [0 for i in range(len(now_data['T'].to_list()))]    
nul_his =  [0 for i in range(len(his_data['T'].to_list()))]
value_now =  now_data['T'].to_list()  + nul_his
value_his = nul_now + his_data['T'].to_list()

CPU times: user 5.63 ms, sys: 1.49 ms, total: 7.11 ms
Wall time: 4.47 ms


In [56]:
len(value_now) 

    def upload2_redis_Hours(self):
        '''根据date_type向redis中传输数据每五分钟传一个数据，并输出对应的键值对'''
        tables = ['TAB_Aws2019','TAB_Mws2019']
        data_list = []
        start,end,daystar = self.current_date()
        # 修改数据层
        start = "2019" + start[4:]
        end = "2019" + end[4:]
        for table in tables:
            sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.lat,station.lon,RR,T,VV,fFy,dFy 
            from {table} as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
            where (tTime between '{start}' and '{end}') order by tTime """
            rsql = sql.format(start=start,end=end,table=table)
            df = pd.read_sql(rsql , con=self.conn) 
            data_list.append(df)
        station_all = pd.concat(data_list)
        # 合并pandas 
        data_class = "table_hours"
        redis_data = self.get_redis(data_class)
        if not redis_data:
            # 当redis无数据时
            data = {
                "data_class":"table_hours",
                "data":station_all
            }
        else:
            # 当redis有数据时,存储数据并保留最近24小时的
            redis_df = redis_data['data']
            redis_df['tTime'] = pd.to_datetime(redis_df['tTime'])
            remain_df = redis_df[redis_df['tTime']>=daystar]
            output = pd.concat([remain_df,station_all])
            output['tTime'] = pd.to_datetime(output['tTime'])
            output.drop_duplicates(keep='first',inplace=True)
            data = {
               "data_class":"table_hours",
                "data":output
            }
        # 将保留的数据重新存储到redis中
        self.rs.set("table_hours", pickle.dumps(data))
        
    def current_date(self):
        '''获取单站数据并解析'''
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        end = today.strftime('%Y-%m-%d %H:%M:%S')
        offset = dtt.timedelta(seconds=-300)
        start = (today + offset).strftime('%Y-%m-%d %H:%M:%S')
        daydelay = dtt.timedelta(days=-1)
        daystar = (today + daydelay).strftime('%Y-%m-%d %H:%M:%S')
        return start,end,daystar
    def decode_dataframe(self,dataframe,decode_type):
        '''根据date_type向redis中获取数据'''
        date_type = "hours24"  
        table_data = []
        data = pickle.loads(self.rs.get(date_type))
        station_all = data['data']  
        grouped_IIiii = station_all.groupby('IIiii')
        for i in grouped_IIiii.size().index:
            single = grouped_IIiii.get_group(i)
            single['tTime'] = pd.to_datetime(single['tTime'])
            end_time = single['tTime'].iloc[-1]
            offset = dtt.timedelta(hours=-decode_type)
            start_time = (end_time + offset)
            start_str = (end_time + offset).strftime('%Y-%m-%d %H:%M:%S')
            end_str = end_time.strftime('%Y-%m-%d %H:%M:%S')
            single_data = single[single['tTime']>start_str]
            # 制定游戏规则
            RR = single_data[single_data['Ri'] > 0.0]['Ri'].sum()
            tmin = single_data['T'].min()
            tmax = single_data['T'].max()
            vv = single_data[(single_data['V'] >0) & (single_data['V'] < 30000)]['V'].min()
            wind = single_data[single_data['fFy'] > 0]['fFy'].max()
            index =  single_data[single_data['fFy'] == single_data['fFy'].max()].index.tolist()[0]
            deg = single_data['dFy'][index]
            single_dir = {
                "IIiii":str(single_data['IIiii'].iloc[0]),
                "county":str(single_data['county'].iloc[0]),
                "town":str(single_data['Town'].iloc[0]),
                "StationName":str(single_data['StationName'].iloc[0]),
                "fFy":str(wind),
                "dFy":str(deg),
                "RR":str(RR),
                "Tx":str(tmax),
                "Tn":str(tmin),
                "VV":str(vv),
                "lat":str(single_data['lat'].iloc[0]),
                "lon":str(single_data['lon'].iloc[0])
            }
            table_data.append(single_dir)

291

# Redis 同步的第三部

In [2]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
from datetime import timezone
class station_zdz:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.rs = redis.Redis(host='127.0.0.1', port=6379)
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
    def time_today(self):
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        return today
    def get_redis(self,date_type):
        '''根据date_type向redis中获取数据'''
        value = self.rs.get(date_type)
        if value:
            data = pickle.loads(value)
        else:
            data = None       
        #data = pickle.loads(self.rs.get(date_type))
        # 解析数据
        return data
    def single_station(self,station):
        '''获取单站数据并解析'''
        # 获取当前时间
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        end = today.strftime('%Y-%m-%d %H:%M:%S')
        offset = dtt.timedelta(days=-1)
        start = (today + offset).strftime('%Y-%m-%d %H:%M:%S')
        # 造假数据
        start = '2019-08-07 20:53:00'
        end = '2019-08-08 20:53:00' 
        yesday = start[0:10] + " 20:00:00"
        today = end[0:10] + " 20:00:00"
        hours = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S').hour
        # 数据库中读取单站数据并解析
        sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.lat,station.lon,Ri,T,V,fFy,dFy 
        from Tab_AM_M as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
        where (ta.IIiii='{station}' and tTime between '{start_time}' and '{end_time}')  order by tTime  """
        rsql = sql.format(start_time=start,end_time=end,station=station)
        data = pd.read_sql(rsql , con=self.conn)
        if hours>=20:
            # 实时数据
            now_data = data[data['tTime']>=today]
            now_data = now_data[now_data['tTime']<=end]
            # 历史数据
            his_data = data[data['tTime']>start]
            his_data = his_data[his_data['tTime']<=today]   
        else:
            # 实时数据
            now_data = data[data['tTime']>=yesday]
            now_data = now_data[now_data['tTime']<=end]
            # 历史数据
            his_data = data[data['tTime']>start]
            his_data = his_data[his_data['tTime']<=yesday]
        # 开始筛选数据种类
        nul_now = [0 for i in range(len(now_data['T'].to_list()))]    
        nul_his =  [0 for i in range(len(his_data['T'].to_list()))]
        value_now =  now_data['T'].to_list()  + nul_his
        value_his = nul_now + his_data['T'].to_list() 
        # 解析数据成两个序列
        return data
    def upload2_redis_Minutes(self):
        '''根据date_type向redis中传输数据'''
        table = 'Tab_AM_M'
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.ZoomLevel,station.Type,station.lat,station.lon,Ri,T,V,fFy,dFy 
        from {table} as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
        where (tTime > '{time}') order by tTime """
        # 数据加载
        data_class = "table_minutes"
        redis_data = self.get_redis(data_class)
        if not redis_data:
            # 当redis无数据时
            #time = today.strftime('%Y-%m-%d %H:%M:%S')
            offset = dtt.timedelta(seconds=-120)
            time = (today + offset).strftime('%Y-%m-%d %H:%M:%S')
            # 测试
            time = '2019-08-09 19:30:00' 
            # 测试
            rsql = sql.format(time=time,table=table)
            station_all = pd.read_sql(rsql , con=self.conn)
            data = {
                'time':time,
                "data_class":"table_minutes",
                "data":station_all
            }
        else:
            # 当redis有数据时,存储数据并保留最近24小时的
            redis_df = redis_data['data']
            time = redis_df['tTime'].iloc[-1].strftime('%Y-%m-%d %H:%M:%S')
            daydelay = dtt.timedelta(days=-1)
            daystar = (today + daydelay).strftime('%Y-%m-%d %H:%M:%S')
            redis_df['tTime'] = pd.to_datetime(redis_df['tTime'])
            # 测试
            time = '2019-08-09 19:30:00'
            daystar = '2019-08-08 19:30:00'
            # 测试
            remain_df = redis_df[redis_df['tTime']>=daystar]
            rsql = sql.format(time=time,table=table)
            station_all = pd.read_sql(rsql , con=self.conn)
            output = pd.concat([remain_df,station_all])
            output['tTime'] = pd.to_datetime(output['tTime'])
            output.drop_duplicates(keep='first',inplace=True)
            data = {
                'time':time,
                "data_class":"table_minutes",
                "data":output
            }
        # 将保留的数据重新存储到redis中
        self.rs.set("table_minutes", pickle.dumps(data))  
    def sql_now(self,decode_type,area):
        '''根据date_type向redis中获取数据'''
        return data
    def decode_time(self,data,times,area,boundary):
        #date_type = "table_minutes"
        #times = 60*3
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        offset = dtt.timedelta(minutes=-times)
        timeindex = (today + offset).strftime('%Y-%m-%d %H:%M:%S')
        # 测试时间
        timeindex = '2019-08-09 19:00:00'
        #data = pickle.loads(self.rs.get(date_type))['data']
        if boundary!="all":
            lat0 =  boundary[0]
            lat1 =  boundary[1]
            lon0 =  boundary[2]
            lon1 =  boundary[3]
        else:
            lat0 = 25
            lat1 = 35
            lon0 = 110
            lon1 = 125
        if area=="nation":
            remain = data[(data['lat']>lat0) & (data['lat']<lat1)  &  (data['lon']<lon1) & (data['lon']>lon0) & (pd.isnull(data['Type']))&(data['ZoomLevel']<6)&(data['tTime']>timeindex)]
        if area=="regin":
            remain = data[(data['lat']>lat0) & (data['lat']<lat1)  &  (data['lon']<lon1) & (data['lon']>lon0) & (data['Type']=="区域站")&(data['tTime']>timeindex)]
        elif area=="all":
            remain = data[(data['lat']>lat0) & (data['lat']<lat1)  &  (data['lon']<lon1) & (data['lon']>lon0)&(data['tTime']>timeindex)]
        elif area=="main":
            remain = data[(data['lat']>lat0) & (data['lat']<lat1)  &  (data['lon']<lon1) & (data['lon']>lon0) & (data['ZoomLevel']<6)&(data['tTime']>timeindex)]
        return remain
    def decode_rain(self,data):
        data = data[data['Ri']>0]
        grouped_IIiii = data.groupby('IIiii')
        table_list = []
        for i in grouped_IIiii.size().index:
            single = grouped_IIiii.get_group(i)
            value = single['Ri'].sum()
            single_data = {
                "IIiii":str(single['IIiii'].iloc[0]),
                "StationName":str(single['StationName'].iloc[0]),
                "Province":str(single['Province'].iloc[0]),
                "City":str(single['City'].iloc[0]),
                "County":str(single['County'].iloc[0]),
                "Town":str(single['Town'].iloc[0]),
                "lat":str(single['lat'].iloc[0]),
                "lon":str(single['lon'].iloc[0]),
                "value":str(value)         
            }
            table_list.append(single_data)
        return table_list
    def decode_tmin(self,data):
        data = data[data['T']!=-9999]
        grouped_IIiii = data.groupby('IIiii')
        table_list = []
        for i in grouped_IIiii.size().index:
            single = grouped_IIiii.get_group(i)
            value = single['T'].min()
            single_data = {
                "IIiii":str(single['IIiii'].iloc[0]),
                "StationName":str(single['StationName'].iloc[0]),
                "Province":str(single['Province'].iloc[0]),
                "City":str(single['City'].iloc[0]),
                "County":str(single['County'].iloc[0]),
                "Town":str(single['Town'].iloc[0]),
                "lat":str(single['lat'].iloc[0]),
                "lon":str(single['lon'].iloc[0]),
                "value":str(value)         
            }
            table_list.append(single_data)
        return table_list
    def decode_tmax(self,data):
        data = data[data['T']!=-9999]
        grouped_IIiii = data.groupby('IIiii')
        table_list = []
        for i in grouped_IIiii.size().index:
            single = grouped_IIiii.get_group(i)
            value = single['T'].max()
            single_data = {
                "IIiii":str(single['IIiii'].iloc[0]),
                "StationName":str(single['StationName'].iloc[0]),
                "Province":str(single['Province'].iloc[0]),
                "City":str(single['City'].iloc[0]),
                "County":str(single['County'].iloc[0]),
                "Town":str(single['Town'].iloc[0]),
                "lat":str(single['lat'].iloc[0]),
                "lon":str(single['lon'].iloc[0]),
                "value":str(value)         
            }
            table_list.append(single_data)
        return table_list
    def decode_view(self,data):
        data = data[data['V']>=0]
        grouped_IIiii = data.groupby('IIiii')
        table_list = []
        for i in grouped_IIiii.size().index:
            single = grouped_IIiii.get_group(i)
            value = single['V'].min()
            single_data = {
                "IIiii":str(single['IIiii'].iloc[0]),
                "StationName":str(single['StationName'].iloc[0]),
                "Province":str(single['Province'].iloc[0]),
                "City":str(single['City'].iloc[0]),
                "County":str(single['County'].iloc[0]),
                "Town":str(single['Town'].iloc[0]),
                "lat":str(single['lat'].iloc[0]),
                "lon":str(single['lon'].iloc[0]),
                "value":str(value)         
            }
            table_list.append(single_data)
        return table_list
    def decode_wind(self,data):
        data = data[data['fFy']!=-9999]
        grouped_IIiii = data.groupby('IIiii')
        table_list = []
        for i in grouped_IIiii.size().index:
            single = grouped_IIiii.get_group(i)
            wind = single['fFy'].max()
            index =  single[single['fFy'] == single['fFy'].max()].index.tolist()[0]
            deg = single['dFy'][index]
            single_data = {
                "IIiii":str(single['IIiii'].iloc[0]),
                "StationName":str(single['StationName'].iloc[0]),
                "Province":str(single['Province'].iloc[0]),
                "City":str(single['City'].iloc[0]),
                "County":str(single['County'].iloc[0]),
                "Town":str(single['Town'].iloc[0]),
                "lat":str(single['lat'].iloc[0]),
                "lon":str(single['lon'].iloc[0]),
                "fFy":str(wind),
                "dFy":str(deg)          
            }
            table_list.append(single_data)
        return table_list
    def decode_data(self,decode_type,times,area,boundary,data_type):
        '''根据前段参数处理数据
        返回列表数据
        decode_type--类型
        data_type--请求的数据类型'''
        date_type = "table_minutes"  
        data = pickle.loads(self.rs.get(date_type))['data']
        if decode_type=='now':
            print("解析当前时刻的降水数据")
            data = self.sql_now()
        elif decode_type=='station':
            print("解析单站的降水数据")
        elif decode_type=='aera':
            print("解析面上的降水数据")
            timedata = self.decode_time(data,times,area,boundary)
            if data_type == "rain":
                output = self.decode_rain(timedata)
            elif data_type == "wind":
                output = self.decode_wind(timedata)
            elif data_type == "tmax":
                output = self.decode_tmax(timedata)
            elif data_type == "tmin":
                output = self.decode_tmin(timedata)
            elif data_type == "view":
                output = self.decode_view(timedata)
        return output


start = '2019-08-08 06:00:00'
end = '2019-08-08 18:00:00'  
date_type = 'table_minutes'
station = '58653'
worker = station_zdz(start,end)
dataframe = None
decode_type = "aera"
times = 60*3
area="main" # nation region all main
boundary = "all"
data_type = "wind"
# worker.decode_dataframe(dataframe,decode_type)
#data = worker.single_station(station)
#worker.upload2_redis_Minutes()
# data = worker.get_redis(date_type)
# data['data']

len(worker.decode_data(decode_type,times,area,boundary,data_type))

解析面上的降水数据
CPU times: user 1.93 s, sys: 321 ms, total: 2.25 s
Wall time: 2.29 s


261

In [24]:
data_class = "table_minutes"
data = worker.get_redis(data_class)['data']

In [40]:
%%time
np.nan
np.isnan(data[(data['lat']>25) & (data['lat']<35)  &  (data['lon']<125) & (data['lon']>110)]['Type'].iloc[0])

CPU times: user 15.9 ms, sys: 241 µs, total: 16.1 ms
Wall time: 13.6 ms


True

In [9]:
data

,tTime,IIiii,StationName,Province,City,County,Town,lat,lon,Ri,T,V,fFy,dFy,ZoomLevel,Type
0,2019-08-09 19:31:00,58454,德清,浙江,湖州,德清县,舞阳街道,30.533333,119.983333,0.0,254,20961,67,3,1.0,NaN
1,2019-08-09 19:31:00,58469,大渔山,浙江,舟山,岱山县,大鱼乡,30.316667,121.950000,0.0,-9999,-9999,-9999,-9999,2.0,NaN
2,2019-08-09 19:31:00,58487,东极,浙江,舟山,普陀区,东极镇,30.183333,122.683333,0.0,-9999,-9999,-9999,-9999,2.0,NaN
3,2019-08-09 19:31:00,58557,义乌,浙江,金华,义乌市,福田街道,29.316667,120.066667,0.0,260,16113,93,1,1.0,NaN
4,2019-08-09 19:31:00,58765,北麂,浙江,温州,瑞安市,北麂乡,27.633333,121.200000,2.0,-9999,-9999,-9999,-9999,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20025,2019-08-09 20:00:00,K9724,半洋礁,浙江,舟山,定海区,双桥街道,30.016667,121.966667,0.0,-9999,5233,213,92,16.0,区域站
20026,2019-08-09 20:00:00,K9739,下三横山,浙江,舟山,嵊泗,菜园镇,30.770000,122.650000,0.0,263,-9999,326,67,16.0,区域站
20027,2019-08-09 20:00:00,K9726,蚊虫山,浙江,舟山,岱山县,秀山乡,30.200000,122.250000,0.0,-9999,3711,259,231,16.0,区域站
20028,2019-08-09 20:00:00,K9728,大五峙,浙江,舟山,定海区,岑港街道,30.216667,121.883333,0.0,-9999,3224,279,83,16.0,区域站


In [25]:
remain = data[(data['lat']>25) & (data['lat']<35)  &  (data['lon']<125) & (data['lon']>110) &(data['V']!=-9999)]

In [26]:
df = remain.groupby(['IIiii','StationName','Province','City','County','Town','Type','lat','lon'])['V'].max().reset_index()

In [27]:
df

,IIiii,StationName,Province,City,County,Town,Type,lat,lon,V
0,58443,长兴,浙江,湖州,长兴县,龙山街道,一般站,31.016667,119.900000,29892
1,58444,临平,浙江,杭州,临平区,临平街道,一般站,30.416667,120.283333,12777
2,58446,安吉,浙江,湖州,安吉县,昌硕街道,一般站,30.633333,119.700000,30000
3,58448,临安,浙江,杭州,临安区,锦南街道,基本站,30.300000,119.733333,30000
4,58449,富阳,浙江,杭州,富阳区,富春街道,一般站,30.050000,119.950000,28963
...,...,...,...,...,...,...,...,...,...,...
326,K9725,鱼腥脑,浙江,舟山,岱山县,高亭镇,区域站,30.333333,121.866667,4508
327,K9726,蚊虫山,浙江,舟山,岱山县,秀山乡,区域站,30.200000,122.250000,5824
328,K9728,大五峙,浙江,舟山,定海区,岑港街道,区域站,30.216667,121.883333,4644
329,K9735,双塘,浙江,舟山,普陀区,六横镇,区域站,29.720000,122.150000,3991


In [107]:
%%time
grouped_IIiii = remain.groupby('IIiii')
for i in grouped_IIiii.size().index:
    single = grouped_IIiii.get_group(i)
    #print(single)

CPU times: user 152 ms, sys: 2.62 ms, total: 154 ms
Wall time: 126 ms


In [101]:
df

In [4]:
# %%time
# df.to_json(orient='records',force_ascii=False)

In [123]:
remain

,tTime,IIiii,StationName,Province,City,County,Town,lat,lon,Ri,T,V,fFy,dFy,ZoomLevel,Type


# 数据同步优化第三步

In [2]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
from datetime import timezone
class station_zdz:
    def __init__(self):
        self.rs = redis.Redis(host='127.0.0.1', port=6379)
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
    def time_today(self):
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        return today
    def get_redis(self,date_type):
        '''根据date_type向redis中获取数据'''
        value = self.rs.get(date_type)
        if value:
            data = pickle.loads(value)
        else:
            data = None       
        #data = pickle.loads(self.rs.get(date_type))
        # 解析数据
        return data
    def single_station(self,station):
        '''获取单站数据并解析'''
        # 获取当前时间
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        end = today.strftime('%Y-%m-%d %H:%M:%S')
        offset = dtt.timedelta(days=-1)
        start = (today + offset).strftime('%Y-%m-%d %H:%M:%S')
        # 造假数据
        start = '2019-08-07 20:53:00'
        end = '2019-08-08 20:53:00' 
        yesday = start[0:10] + " 20:00:00"
        today = end[0:10] + " 20:00:00"
        hours = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S').hour
        # 数据库中读取单站数据并解析
        sql = """select tTime,IIiii,Ri,T,V,fFy,dFy 
        from Tab_AM_M where ( tTime between '{start_time}' and '{end_time}' and IIiii='{station}')  order by tTime  """
        rsql = sql.format(start_time=start,end_time=end,station=station)
        data = pd.read_sql(rsql , con=self.conn)
        if hours>=20:
            # 实时数据
            now_data = data[data['tTime']>=today]
            now_data = now_data[now_data['tTime']<=end]
            # 历史数据
            his_data = data[data['tTime']>start]
            his_data = his_data[his_data['tTime']<=today]   
        else:
            # 实时数据
            now_data = data[data['tTime']>=yesday]
            now_data = now_data[now_data['tTime']<=end]
            # 历史数据
            his_data = data[data['tTime']>start]
            his_data = his_data[his_data['tTime']<=yesday]
        # 开始筛选数据种类
#         nul_now = [0 for i in range(len(now_data['T'].to_list()))]    
#         nul_his =  [0 for i in range(len(his_data['T'].to_list()))]
#         value_now =  now_data['T'].to_list()  + nul_his
#         value_his = nul_now + his_data['T'].to_list() 
        output = pd.concat([now_data,his_data])  
        # 解析数据成两个序列
        return data
    def upload2_redis_Minutes(self):
        '''根据date_type向redis中传输数据'''
        table = 'Tab_AM_M'
        SHA_TZ = timezone(
            dtt.timedelta(hours=8),
            name='Asia/Shanghai',
        )
        utc_now = dtt.datetime.utcnow().replace(tzinfo=dtt.timezone.utc)
        today = utc_now.astimezone(SHA_TZ)
        sql = """select tTime,ta.IIiii,station.StationName,station.Province,station.City,station.County,station.Town,station.ZoomLevel,station.Type,station.lat,station.lon,Ri,T,V,fFy,dFy 
        from {table} as ta inner join TAB_StationInfo as station on ta.IIiii=station.IIiii and station.Province ='浙江' 
        where (tTime > '{time}') order by tTime """
        # 数据加载
        data_class = "table_minutes"
        redis_data = self.get_redis(data_class)
        if not redis_data:
            # 当redis无数据时
            #time = today.strftime('%Y-%m-%d %H:%M:%S')
            offset = dtt.timedelta(seconds=-120)
            time = (today + offset).strftime('%Y-%m-%d %H:%M:%S')
            # 测试
            time = '2019-08-09 19:30:00' 
            # 测试
            rsql = sql.format(time=time,table=table)
            station_all = pd.read_sql(rsql , con=self.conn)
            data = {
                'time':time,
                "data_class":"table_minutes",
                "data":station_all
            }
        else:
            # 当redis有数据时,存储数据并保留最近24小时的
            redis_df = redis_data['data']
            time = redis_df['tTime'].iloc[-1].strftime('%Y-%m-%d %H:%M:%S')
            daydelay = dtt.timedelta(days=-1)
            daystar = (today + daydelay).strftime('%Y-%m-%d %H:%M:%S')
            redis_df['tTime'] = pd.to_datetime(redis_df['tTime'])
            # 测试 基本数据
            time = '2019-08-09 19:30:00'
            daystar = '2019-08-08 19:30:00'
            # 测试
            remain_df = redis_df[redis_df['tTime']>=daystar]
            rsql = sql.format(time=time,table=table)
            station_all = pd.read_sql(rsql , con=self.conn)
            output = pd.concat([remain_df,station_all])
            output['tTime'] = pd.to_datetime(output['tTime'])
            output.drop_duplicates(keep='first',inplace=True)
            data = {
                'time':time,
                "data_class":"table_minutes",
                "data":output
            }
        # 将保留的数据重新存储到redis中
        self.rs.set("table_minutes", pickle.dumps(data))  
    def sql_now(self,decode_type,area):
        '''根据date_type向redis中获取数据'''
        return data
    def decode_time(self):
        date_type = "table_minutes"
        data = self.get_redis(date_type)['data']
        times = [60*24,60*12,60*6,60*3,60*2,60*1,45,30,15,5]
        tables = ['24hours','12hours','6hours','3hours','2hours','1hours','45mins','30mins','15mins','5mins']
        # today = self.time_today()
        # 测试时间数据
        end = '2019-08-09 20:00:00' 
        today = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S')
        value_list = ['Ri','Tx','Tn','fFy','V']
        for i, value in enumerate(times):
            offset = dtt.timedelta(minutes=-value)         
            timeindex = (today + offset).strftime('%Y-%m-%d %H:%M:%S')
            remain = data[data['tTime']>timeindex]
            table = tables[i]
            table_data_list = []
            for j in value_list:
                value_df = j
                df = self.return_data(remain,value_df)
                table_data_list.append(df)  
            data_redis = {
                "data_class":tables[i],
                "value_list":value_list,
                "table_data_list":table_data_list
            }
            # 将保留的数据重新存储到redis中
            self.rs.set(tables[i], pickle.dumps(data_redis))  
    def get_regin(self,boundary,table_type,tables_name,value_index):
        '''解码不同类型的数据'''
        data = self.get_redis(tables_name)['table_data_list'][value_index]
        lat0 = boundary[0]
        lat1 = boundary[1]
        lon0 = boundary[2]
        lon1 = boundary[3]
        boundary_data =  data[(data['lat']>lat0) & (data['lat']<lat1)  &  (data['lon']<lon1) & (data['lon']>lon0)]
        if table_type=="nation":
            remain = data[(pd.isnull(data['Type']))&(data['ZoomLevel']<6)]
        if table_type=="regin":
            remain = data[(data['Type']=="区域站")]
        elif table_type=="all":
            remain = data
        elif table_type=="main":
            remain = data[(data['ZoomLevel']<6)]
        output = remain.to_json(orient='records',force_ascii=False)
        return output
    def return_data(self,remain,value):
        '''返回数据的类型
        boundary 边界
        table redis的表
        value 要素 Ri fFy Tx Tn V  
        table_type  nation regin main all 
        '''
        if value=="Ri":
            re = remain[remain['Ri']>0]
            df = re.groupby(['IIiii','StationName','Province','City','County','Town','Type','lat','lon','ZoomLevel'])['Ri'].sum().reset_index()
            df['value'] = df['Ri']
            output = df.to_json(orient='records',force_ascii=False)
        elif value=="fFy":
            re = remain[remain['fFy']>0]
            #output = re
            grouped_IIiii = re.groupby(['IIiii','StationName','Province','City','County','Town','Type','lat','lon','ZoomLevel'])
            all_list = []
            for i in grouped_IIiii.size().index:
                single = grouped_IIiii.get_group(i)
                value = single[ single['fFy']== single['fFy'].max()].head(1)
                all_list.append(value)
            df = pd.concat(all_list) 
            df['value'] = df['fFy']
            output = df.to_json(orient='records',force_ascii=False)         
        elif value=="Tx":
            re = remain[remain['T']!=-9999]
            df = re.groupby(['IIiii','StationName','Province','City','County','Town','Type','lat','lon','ZoomLevel'])['T'].max().reset_index() 
            df['value'] = df['T']
            output = df.to_json(orient='records',force_ascii=False)
        elif value=="Tn":
            re = remain[remain['T']!=-9999]
            df = re.groupby(['IIiii','StationName','Province','City','County','Town','Type','lat','lon','ZoomLevel'])['T'].min().reset_index()
            df['value'] = df['T']
            output = df.to_json(orient='records',force_ascii=False)
        elif value=="V":
            re = remain[remain['V']!=-9999]
            df = re.groupby(['IIiii','StationName','Province','City','County','Town','Type','lat','lon','ZoomLevel'])['V'].min().reset_index() 
            df['value'] = df['V']
            output = df.to_json(orient='records',force_ascii=False)
        return df
        
        
            

start = '2019-08-08 06:00:00'
end = '2019-08-08 18:00:00'  
date_type = 'table_minutes'
station = '58653'
worker = station_zdz()
dataframe = None
decode_type = "aera"
times = 60*3
area="main" # nation region all main
boundary = "all"
data_type = "wind"
# worker.decode_dataframe(dataframe,decode_type)
#data = worker.single_station(station)
# worker.upload2_redis_Minutes()
# data = worker.get_redis(date_type)
# data['data']
# worker.decode_time()
#len(worker.decode_data(decode_type,times,area,boundary,data_type))

CPU times: user 272 ms, sys: 49.8 ms, total: 322 ms
Wall time: 325 ms


In [3]:
%%time
worker.decode_time()

CPU times: user 40.7 s, sys: 682 ms, total: 41.4 s
Wall time: 41.3 s


In [63]:
%%time
# data_redis = {
#     "data_class":tables[i],
#     "value_list":value_list,
#     "table_data_list":table_data_list
# }
date_type = '15mins'
# date_type = 'table_minutes'
data = worker.get_redis(date_type)

CPU times: user 11.9 ms, sys: 58 µs, total: 12 ms
Wall time: 10.1 ms


In [64]:
data['table_data_list']

[      IIiii StationName Province City County  Town Type        lat  \
 0     58443          长兴       浙江   湖州    长兴县  龙山街道  一般站  31.016667   
 1     58444          临平       浙江   杭州    临平区  临平街道  一般站  30.416667   
 2     58446          安吉       浙江   湖州    安吉县  昌硕街道  一般站  30.633333   
 3     58449          富阳       浙江   杭州    富阳区  富春街道  一般站  30.050000   
 4     58450          湖州       浙江   湖州    湖州市  龙溪街道  基本站  30.850000   
 ...     ...         ...      ...  ...    ...   ...  ...        ...   
 2483  K9711         沼潭村       浙江   舟山    岱山县   衢山镇  区域站  30.433333   
 2484  K9714         沈家湾       浙江   舟山    嵊泗县   洋山镇  区域站  30.600000   
 2485  K9719         泥螺山       浙江   舟山    岱山县   衢山镇  区域站  30.416667   
 2486  K9735          双塘       浙江   舟山    普陀区   六横镇  区域站  29.720000   
 2487  K9739        下三横山       浙江   舟山     嵊泗   菜园镇  区域站  30.770000   
 
              lon  ZoomLevel    Ri  value  
 0     119.900000          1  24.0   24.0  
 1     120.283333          2  27.0   27.0  
 2     119.700

In [12]:
data = worker.single_station(station)
data['tTime'] = pd.to_datetime(data['tTime'])

In [44]:
%%time
data['Year'] = data['tTime'].dt.year
data['Month'] = data['tTime'].dt.month
data['Day'] = data['tTime'].dt.day
data['Hour'] = data['tTime'].dt.hour
grouped_IIiii = data.groupby(['Day','Hour'])
time_list = []
dir_list = []
speed_list = []
for i in grouped_IIiii.size().index:
    single = grouped_IIiii.get_group(i)
    value = single[ single['fFy']== single['fFy'].max()].head(1)
    time_list.append(value['tTime'].dt.strftime('%Y-%m-%d %H:%M:%S').values[0][11:16])
    speed_list.append(value['fFy'].values[0])
    dir_list.append(value['dFy'].values[0])

# df = pd.concat(all_list) 
# df    
# print(grouped_IIiii.size().index)

CPU times: user 39.5 ms, sys: 9.24 ms, total: 48.8 ms
Wall time: 42.3 ms


In [3]:
%%time
# data_redis = {
#     "data_class":tables[i],
#     "value_list":value_list,
#     "table_data_list":table_data_list
# }
date_type = '24hours'
# date_type = 'table_minutes'
data = worker.get_redis(date_type)

CPU times: user 18.8 ms, sys: 0 ns, total: 18.8 ms
Wall time: 17.2 ms


In [7]:
boundary_data = data['table_data_list'][0]
remain = boundary_data[(boundary_data['ZoomLevel']<6)]


In [10]:
remain[remain['Type']=='基本站']

,IIiii,StationName,Province,City,County,Town,Type,lat,lon,ZoomLevel,Ri,value
3,58448,临安,浙江,杭州,临安区,锦南街道,基本站,30.300000,119.733333,1,362.0,362.0
5,58450,湖州,浙江,湖州,湖州市,龙溪街道,基本站,30.850000,120.083333,1,154.0,154.0
15,58464,平湖,浙江,嘉兴,平湖市,乍浦镇,基本站,30.650000,121.116667,1,222.0,222.0
17,58467,慈溪,浙江,宁波,慈溪市,白沙路街道,基本站,30.200000,121.300000,1,294.0,294.0
21,58472,嵊泗,浙江,舟山,嵊泗县,菜园镇,基本站,30.733333,122.450000,1,499.0,499.0
29,58543,淳安,浙江,杭州,淳安县,千岛湖镇,基本站,29.616667,119.016667,1,151.0,151.0
34,58549,金华,浙江,金华,金华市,城中街道,基本站,29.116667,119.650000,1,47.0,47.0
36,58553,上虞,浙江,绍兴,上虞区,崧厦镇,基本站,30.050000,120.816667,1,413.0,413.0
39,58556,嵊州,浙江,绍兴,嵊州市,剡湖街道,基本站,29.600000,120.816667,1,175.0,175.0
40,58557,义乌,浙江,金华,义乌市,福田街道,基本站,29.316667,120.066667,1,119.0,119.0


In [17]:
data['value_list']

['Ri', 'Tx', 'Tn', 'fFy', 'V']

In [5]:
%%time
data = worker.single_station(station)

CPU times: user 11.8 ms, sys: 13.1 ms, total: 24.9 ms
Wall time: 3.38 s


In [ ]:
data.to_json(orient='values',force_ascii=False)

In [43]:
%%time
worker.decode_time()

CPU times: user 27 s, sys: 861 ms, total: 27.8 s
Wall time: 27.8 s


In [28]:
%%time
table,boundary,value,table_type,tables_name,value_index = '3hours',[25,35,115,125],'Tx','main','12hours',1
wind = worker.get_regin(boundary,table_type,tables_name,value_index)

CPU times: user 35 ms, sys: 40 µs, total: 35 ms
Wall time: 31.4 ms


In [29]:
wind

,IIiii,StationName,Province,City,County,Town,Type,lat,lon,ZoomLevel,T,value
0,58443,长兴,浙江,湖州,长兴县,龙山街道,一般站,31.016667,119.900000,1,331,331
1,58444,临平,浙江,杭州,临平区,临平街道,一般站,30.416667,120.283333,2,321,321
2,58446,安吉,浙江,湖州,安吉县,昌硕街道,一般站,30.633333,119.700000,1,324,324
3,58448,临安,浙江,杭州,临安区,锦南街道,基本站,30.300000,119.733333,1,308,308
4,58449,富阳,浙江,杭州,富阳区,富春街道,一般站,30.050000,119.950000,1,326,326
...,...,...,...,...,...,...,...,...,...,...,...,...
2798,K9313,牛角圩,浙江,丽水,松阳县,大东坝镇,区域站,28.350000,119.500000,4,284,284
2799,K9314,大岭脚,浙江,丽水,松阳县,玉岩镇,区域站,28.400000,119.300000,4,275,275
2802,K9317,戈溪,浙江,丽水,青田县,船寮镇,区域站,28.266667,120.150000,4,300,300
2830,K9345,沈坑,浙江,丽水,景宁县,渤海镇,区域站,28.050000,119.766667,4,271,271


In [15]:
wind

,tTime,IIiii,StationName,Province,City,County,Town,ZoomLevel,Type,lat,lon,Ri,T,V,fFy,dFy
42820,2019-08-08 21:01:00,58459,萧山,浙江,杭州,萧山区,北干街道,1,一般站,30.183333,120.283333,0.0,-9999,-9999,-9999,-9999
42821,2019-08-08 21:01:00,58537,开化,浙江,衢州,开化县,芹阳办事处,1,一般站,29.133333,118.400000,0.0,311,20000,27,81
42822,2019-08-08 21:01:00,58564,白石山,浙江,宁波,宁海县,None,2,区域站,29.266667,121.500000,0.0,-9999,-9999,-9999,-9999
42823,2019-08-08 21:01:00,58569,石浦,浙江,宁波,象山县,石浦镇,1,基本站,29.200000,121.950000,0.0,277,12257,129,50
42824,2019-08-08 21:01:00,58633,衢州,浙江,衢州,衢州衢江区,浮石街道,1,基本站,29.000000,118.900000,0.0,297,20000,59,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871224,2019-08-09 20:00:00,K8818,佃石,浙江,台州,三门县,亭旁镇,4,区域站,29.002500,121.314700,17.0,-9999,-9999,-9999,12
871227,2019-08-09 20:00:00,K9003,小梅,浙江,丽水,龙泉市,小梅镇,4,区域站,27.816667,118.966667,0.0,247,-9999,24,234
871230,2019-08-09 20:00:00,K9041,大漈,浙江,丽水,景宁县,大漈乡,4,区域站,27.800000,119.600000,2.0,208,-9999,140,290
871241,2019-08-09 20:00:00,K9232,祯埠,浙江,丽水,青田县,祯埠镇,4,区域站,28.316667,120.016667,1.0,264,-9999,88,114


In [129]:
wind1 = wind.groupby(['IIiii','StationName','Province','City','County','Town','Type','lat','lon'])

In [134]:
%%time
all_list = []
for i in wind1.size().index:
    single = wind1.get_group(i)
    dfy = single[ single['fFy']== single['fFy'].max()].head(1)
    all_list.append(dfy)
    
alldata = pd.concat(all_list)
alldata

CPU times: user 560 ms, sys: 33 ms, total: 593 ms
Wall time: 523 ms


,tTime,IIiii,StationName,Province,City,County,Town,ZoomLevel,Type,lat,lon,Ri,T,V,fFy,dFy
825871,2019-08-09 18:55:00,58443,长兴,浙江,湖州,长兴县,龙山街道,1,一般站,31.016667,119.900000,0.0,273,26333,131,67
858747,2019-08-09 19:45:00,58444,临平,浙江,杭州,临平区,临平街道,2,一般站,30.416667,120.283333,1.0,253,12777,135,73
798905,2019-08-09 18:03:00,58446,安吉,浙江,湖州,安吉县,昌硕街道,1,一般站,30.633333,119.700000,7.0,270,6711,114,50
835463,2019-08-09 19:09:00,58448,临安,浙江,杭州,临安区,锦南街道,1,基本站,30.300000,119.733333,0.0,249,14454,167,26
801353,2019-08-09 18:10:00,58449,富阳,浙江,杭州,富阳区,富春街道,1,一般站,30.050000,119.950000,1.0,258,15500,175,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834925,2019-08-09 19:05:00,K9313,牛角圩,浙江,丽水,松阳县,大东坝镇,4,区域站,28.350000,119.500000,0.0,266,-9999,80,22
867546,2019-08-09 19:55:00,K9314,大岭脚,浙江,丽水,松阳县,玉岩镇,4,区域站,28.400000,119.300000,6.0,237,-9999,106,0
808933,2019-08-09 18:20:00,K9317,戈溪,浙江,丽水,青田县,船寮镇,4,区域站,28.266667,120.150000,1.0,267,-9999,111,258
841386,2019-08-09 19:15:00,K9345,沈坑,浙江,丽水,景宁县,渤海镇,4,区域站,28.050000,119.766667,3.0,237,-9999,128,264


In [105]:
for i in wind1.size().index:
    single = wind1.get_group(i)
single

,tTime,IIiii,StationName,Province,City,County,Town,ZoomLevel,Type,lat,lon,Ri,T,V,fFy,dFy
803564,2019-08-09 18:10:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,25.0,251,1546,95,112
808605,2019-08-09 18:20:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,6.0,250,2735,99,299
815261,2019-08-09 18:35:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,29.0,249,2180,100,287
818854,2019-08-09 18:40:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,19.0,250,1247,138,116
821962,2019-08-09 18:45:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,14.0,251,2048,146,80
824425,2019-08-09 18:50:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,4.0,253,3200,166,75
828308,2019-08-09 18:55:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,4.0,254,3141,167,72
831640,2019-08-09 19:00:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,2.0,255,4442,177,75
834968,2019-08-09 19:05:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,2.0,255,5134,183,70
837238,2019-08-09 19:10:00,K9548,沥港,浙江,舟山,定海区,金塘镇,4,区域站,30.066667,121.816667,1.0,255,3493,186,74


In [88]:
wind1.groupby(['IIiii','StationName','Province','City','County','Town','Type','lat','lon'])['fFy','dFy'].max()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,,,,,,,,,fFy,dFy
IIiii,StationName,Province,City,County,Town,Type,lat,lon,,
58443,长兴,浙江,湖州,长兴县,龙山街道,一般站,31.016667,119.900000,131,84
58444,临平,浙江,杭州,临平区,临平街道,一般站,30.416667,120.283333,135,357
58446,安吉,浙江,湖州,安吉县,昌硕街道,一般站,30.633333,119.700000,114,358
58448,临安,浙江,杭州,临安区,锦南街道,基本站,30.300000,119.733333,167,351
58449,富阳,浙江,杭州,富阳区,富春街道,一般站,30.050000,119.950000,175,357
...,...,...,...,...,...,...,...,...,...,...
K9313,牛角圩,浙江,丽水,松阳县,大东坝镇,区域站,28.350000,119.500000,80,340
K9314,大岭脚,浙江,丽水,松阳县,玉岩镇,区域站,28.400000,119.300000,106,0
K9317,戈溪,浙江,丽水,青田县,船寮镇,区域站,28.266667,120.150000,111,278


In [138]:
wind[(wind['IIiii']=='58443')&(wind['fFy']==131)]

,tTime,IIiii,StationName,Province,City,County,Town,ZoomLevel,Type,lat,lon,Ri,T,V,fFy,dFy
825871,2019-08-09 18:55:00,58443,长兴,浙江,湖州,长兴县,龙山街道,1,一般站,31.016667,119.9,0.0,273,26333,131,67


In [95]:
wind1.iloc[wind1[(wind1['IIiii']=='58444')&(wind1['fFy']==135)].index]

,IIiii,StationName,Province,City,County,Town,Type,lat,lon,dFy,fFy
41,58444,临平,浙江,杭州,临平区,临平街道,一般站,30.416667,120.283333,73,135


In [78]:
wind[wind['IIiii']=='58443']['fFy'].max()

131

In [34]:
%%time
# date_type = '15minutes'
date_type = 'table_minutes'
data = worker.get_redis(date_type)['data']
# worker.decode_time()

# data['tTime'] = pd.to_datetime(data['tTime'])

CPU times: user 2.01 s, sys: 492 ms, total: 2.51 s
Wall time: 2.57 s


In [38]:
%%time
worker.decode_time()

CPU times: user 9.79 s, sys: 935 ms, total: 10.7 s
Wall time: 10.9 s


In [47]:
%%time
# date_type = '3minutes'
date_type = '3hours'
worker.get_redis(date_type)['data']

CPU times: user 210 ms, sys: 30.1 ms, total: 240 ms
Wall time: 243 ms


,tTime,IIiii,StationName,Province,City,County,Town,ZoomLevel,Type,lat,lon,Ri,T,V,fFy,dFy
798723,2019-08-09 18:01:00,58459,萧山,浙江,杭州,萧山区,北干街道,1,一般站,30.183333,120.283333,0.0,-9999,-9999,-9999,-9999
798724,2019-08-09 18:01:00,58537,开化,浙江,衢州,开化县,芹阳办事处,1,一般站,29.133333,118.400000,1.0,-9999,-9999,-9999,-9999
798725,2019-08-09 18:01:00,58569,石浦,浙江,宁波,象山县,石浦镇,1,基本站,29.200000,121.950000,2.0,255,1469,261,51
798726,2019-08-09 18:01:00,58633,衢州,浙江,衢州,衢州衢江区,浮石街道,1,基本站,29.000000,118.900000,0.0,287,20000,41,42
798727,2019-08-09 18:01:00,58452,嘉兴,浙江,嘉兴,嘉兴市,长水街道,1,一般站,30.733333,120.750000,0.0,-9999,-9999,-9999,-9999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871264,2019-08-09 20:00:00,K9724,半洋礁,浙江,舟山,定海区,双桥街道,16,区域站,30.016667,121.966667,0.0,-9999,5233,213,92
871265,2019-08-09 20:00:00,K9739,下三横山,浙江,舟山,嵊泗,菜园镇,16,区域站,30.770000,122.650000,0.0,263,-9999,326,67
871266,2019-08-09 20:00:00,K9726,蚊虫山,浙江,舟山,岱山县,秀山乡,16,区域站,30.200000,122.250000,0.0,-9999,3711,259,231
871267,2019-08-09 20:00:00,K9728,大五峙,浙江,舟山,定海区,岑港街道,16,区域站,30.216667,121.883333,0.0,-9999,3224,279,83


# 单站